In [5]:
import nibabel as nib
import pydicom
import numpy as np
import os
from pydicom.dataset import Dataset, FileDataset
from datetime import datetime
from pydicom.uid import generate_uid
import matplotlib.pyplot as plt
import requests
import simple_orthanc




def delete_series(series_to_delete, orthanc_url='http://localhost:8042'):
    series_url = f"{orthanc_url}/series/"
    params = {}
    params['expand'] = 1
    params['requestedTags'] = "SeriesInstanceUID"
    
    series_response = requests.get(series_url, params)

    series_id = [entry['ID'] for entry in series_response.json() if entry.get('RequestedTags', {}).get('SeriesInstanceUID') == series_to_delete]
    for serie_id in series_id:
        serie_response = requests.delete(f"{orthanc_url}/series/"+serie_id)
        if serie_response.status_code != 200:
            print(f"Failed to retrieve patient information. Status code: {serie_response.status_code}")
            print(f"Response: {serie_response.text}")
            return
        print(f"Deleted Study: {serie_id}")


def upload_dicom_folder(dicom_folder):
    # Initialize the Orthanc client
    print("Uploading DICOM folder to Orthanc...")
    orthanc = simple_orthanc.Orthanc()

    
    orthanc.upload_folder(dicom_folder, test_dicom=False, recursive=False)





def nifti_to_dicom(nifti_file,
    series_description,
    series_instance_uid, # should be different for each image
    reference_dicom_file = "../data/dicom/real/extensive-consolidation-v2-sampl/image.0001.dcm",
    modality='CT',
    study_instance_uid = '1', # should be the same for each study (for AI/non-AI)
    study_id='1', # should be the same for AI/non-AI
    patient_name="Generative AI Patient",
    patient_id="MedSyn",
    description="",

    ):
    output_folder = nifti_file.split(".nii.gz")[0] # stores in folder with same name as input file
    print(f"Store DICOM files in Folder: {output_folder}")
    rotate=""
    
    
    if modality=='AI':
        rotate="counterclockwise"
    else:
        rotate="clockwise"

    # Load the NIfTI file
    img = nib.load(nifti_file)
    data = img.get_fdata()
    affine = img.affine

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize common dataset attributes
    ds = pydicom.dcmread(reference_dicom_file)
    ds.PatientName = patient_name
    ds.PatientID = patient_id
    ds.AccessionNumber = study_instance_uid
    ds.StudyInstanceUID = study_instance_uid
    ds.SeriesInstanceUID = series_instance_uid
    ds.SeriesDescription= series_description
    ds.Modality = modality
    ds.SeriesNumber = 1
    ds.StudyID = study_id
    ds.StudyDescription = description
    ds.Manufacturer = "PythonDicomConversion"
    ds.Rows, ds.Columns = data.shape[:2]
    ds.SliceThickness = float(affine[2, 2])

    ds.SamplesPerPixel = 1
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15
    ds.PixelRepresentation = 1  # 1 means signed integers




    # Set additional metadata
    ds.ContentDate = str(datetime.now().date()).replace('-', '')
    ds.ContentTime = datetime.now().strftime("%H%M")

    ds.StudyDate = ds.ContentDate
    ds.StudyTime = ds.ContentTime
    ds.PatientSex = "O"
    ds.PatientBirthDate = "19000101"


    # Scale pixel data if necessary (e.g., to avoid issues with pixel value ranges)
    #data = data - np.min(data)
    #data = data / np.max(data) * (3000)
    
    # norm the HUE data between -2000, and 2000
    data = (data - np.min(data)) / (np.max(data) - np.min(data)) * 4000 - 2000
    data = data.astype('int16')

    # reverse in 3rd axis
    #data = data[:,:,::-1]
    # Rotate each slice to the left (90 degrees counterclockwise)
    if rotate == "counterclockwise":
        data = np.rot90(data, k=1, axes=(0, 1))
    elif rotate == "clockwise":
        data = np.rot90(data, k=3, axes=(0, 1))
        
    
    #print(data)
    # Iterate over each slice and update the dataset
    for i in range(data.shape[2]):
        slice_data = data[:, :, -(i+1)]

        # Update slice-specific attributes
        ds.SOPInstanceUID= generate_uid()
        ds.InstanceNumber = i + 1
        ds.ImagePositionPatient = [0,0,-i] 
        ds.SliceLocation = i * ds.SliceThickness



        # Convert pixel data to the appropriate type and flatten the array
        ds.PixelData = slice_data.tobytes()

        # Visualize the slice
        # plt.imshow(slice_data, cmap='gray')
        # plt.title(f'Slice {i}')
        # plt.show()

        # Save the DICOM file
        dicom_filename = os.path.join(output_folder, f"slice_{i:03d}.dcm")
        ds.save_as(dicom_filename)

    print(f"Conversion complete. DICOM files are saved in {output_folder}")

    # delete if there is already a study for this patient
    delete_series(series_to_delete=series_instance_uid)
    
    upload_dicom_folder(output_folder)
    print("Files Uploaded to Orthanc Server Localhost")


In [8]:
import json

def store_metadata(series_instance_uid, metadata, json_file_path="../data/init_metadata.json"):
    # Load the existing JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
        
    # Ensure the series_instance_uid exists in the data
    if series_instance_uid not in data:
        data[series_instance_uid] = {}
    # Add or override the entry
    for key, value in metadata.items():
        data[series_instance_uid][key] = value

    # Write the updated data back to the JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"Entry with study_instance_uid {series_instance_uid} added or updated.")

In [3]:
import requests

def _get_orthanc_study_id(study_instance_uid):
    try:
        # Parameters to include in the request
        params = {
            'expand': 1,
            'requestedTags': 'StudyInstanceUID'
        }

        # Fetching DICOM studies from the PACS server with query parameters
        response = requests.get('http://localhost/pacs/studies', params=params)
        
        # Check if the response is ok (status code 200-299)
        if response.status_code != 200:
            print(f"Network response was not ok. Status: {response.status_code}")
            return None
        
        # Parse the response as JSON
        data = response.json()

        # Filter the data to find the study with the given StudyInstanceUID
        study = next((item for item in data if item['RequestedTags']['StudyInstanceUID'] == study_instance_uid), None)

        # Check if the study was found
        if study:
            return study['ID']
        else:
            return None
    except requests.exceptions.RequestException as e:
        # Log any errors that occur during the fetch operation
        print(f'There has been a problem with your fetch operation: {e}')
        return None
    
def add_impression_metadata_to_study(study_instance_uid, data):
    study_id = _get_orthanc_study_id(study_instance_uid)
    try:
        url = f'http://localhost/pacs/studies/{study_id}/metadata/Impressions'
        headers = {
            'Content-Type': 'text/plain'  # Ensure the server expects text/plain content type
        }
        
        response = requests.put(url, headers=headers, data=data)

        if response.status_code != 200:
            print(f"Response not ok. Status: {response.status_code}, Response text: {response.text}")
            return

    except requests.exceptions.RequestException as e:
        print(f'There was a problem with your fetch operation: {e}')


#add_impression_metadata_to_study('1.1', 'Test')


# CT-Rate


## Example 1
- Arterial wall calcification
- Atelectasis
- Lung opacity
- Consolidation

### Original


In [9]:
findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_1_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file,
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.1',
               series_instance_uid='1.1.1',
               study_id='1', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
               )
store_metadata(series_instance_uid='1.1',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.1', impressions)



Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1_a_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1_a_1
Deleted Study: 9a4454ac-bc732adf-25c4a3af-10c77925-9de83335
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1 added or updated.


### Generated

In [62]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.2',
               study_id='2', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
store_metadata(series_instance_uid='1.1.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_0
Deleted Study: 97265e43-0d353c37-3d27a9a5-05c2541d-fdd188b8
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1.2 added or updated.


In [63]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.3',
               study_id='3', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
store_metadata(series_instance_uid='1.1.3',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_1
Deleted Study: 35fd8b02-62d06425-325bf754-a4a044d8-9f0364b0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1.3 added or updated.


In [64]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_2.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 3", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.4',
               study_id='4', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
store_metadata(series_instance_uid='1.1.4',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_2
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_2
Deleted Study: 4be1d0e3-f5ab6fdc-24f2df0c-7f6cfe46-997fdaf4
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1.4 added or updated.


### Counterfactual

In [65]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  No thickening of the bronchial wall.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 4 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.5',
               study_id='5',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
store_metadata(series_instance_uid='1.1.5',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_1_sample_0
Deleted Study: 81585060-7833783d-9a089a6c-7c9d2fda-7be1ea99
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1.5 added or updated.


In [66]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, large consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_2_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 5 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.6',
               study_id='6',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
store_metadata(series_instance_uid='1.1.6',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_2_sample_0
Deleted Study: c772418f-783ee516-97dd3ff8-64d8da22-16814d47
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.1.6 added or updated.


## Example 2
- Arterial wall calcification
- Coronary artery wall calcification
- Lymphadenopathy
- Lung nodule
- Lung opacity


### Original 

In [10]:

findings = "Trachea, both main bronchi are open. Mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Calcific plaques are present in the coronary arteries. Thoracic esophagus calibration was normal and no significant pathological wall thickening was detected. Millimetric lymph nodes with a short axis not exceeding 1 cm are observed in the mediastinum. When examined in the lung parenchyma window; Widespread ground glass densities are observed in both lung parenchyma showing a tendency to central and peripheral fusion. A 5 mm calcific nodule was observed in the posterobasal region of the lower lobe of the right lung. A millimetric hypodense lesion was observed in segment 2 of the liver in the upper abdominal organs included in the sections. Bone structures in the study area are natural. Osteodegenerative changes are observed in the vertebrae."
impressions="Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Arterial wall calcification, Coronary artery wall calcification, Lymphadenopathy, Lung nodule, Lung opacity"
nifti_file = '../data/nifti/ct-rate/train/train_10_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.2',
               series_instance_uid='1.2.1',
               study_id='1',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.2',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.2', impressions)

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_10_a_1


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (107) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_10_a_1
Deleted Study: af503a5b-a3ebfdfb-a362f881-f5808707-cff51ae8
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.2 added or updated.


### Generated

In [40]:
prompt=" Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Arterial wall calcification, Coronary artery wall calcification, Lymphadenopathy, Lung nodule, Lung opacity"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_10_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.2',
               series_instance_uid='1.2.2',
               study_id='2',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,

               )
store_metadata(series_instance_uid='1.2.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_10_a_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_10_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.2.2 added or updated.


# Example 3
- (Medical material)
- Pericardial effusion
- Emphysema
- Atelectasis
- Lung nodule
- Pleural effusion
- Consolidation


### Original 

In [11]:

findings = "A central venous catheter inserted from the right was observed. Trachea and both main bronchi are open and no obstructive pathology is detected. Mediastinal vascular structures and cardiac examination could not be evaluated optimally due to the lack of contrast. The pulmonary trunk caliber was measured at 30 mm and was wider than normal. Heart contour, the size is natural. There is minimal pericardial effusion. The effusion measuring approximately 80 mm in size is observed on the right. There is an area of increased density in the lung parenchyma adjacent to the effusion, which is considered secondary to compressive atelectasis. No pathological increase in wall thickness was observed in the thoracic esophagus. In the mediastinum, in both axillary regions and in the supraclavicular fossa, no lymph nodes were observed in pathological size and appearance. In the lower lobe of the left lung, there is an area of increase in density consistent with the consolidation observed in the air bronchograms. Although the appearance may be secondary to atelectasis, underlying pneumonic infiltration cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings. No mass lesion was observed in both lungs. There are a few millimetric nodules in the left lung, the largest of which is 5.5x3.5 mm in the upper lobe inferior lingular segment. Emphysematous changes were observed in both lungs. There are findings consistent with peritoneal carcinomatosis in the upper abdominal sections within the image, and hypodense lesions in the liver and spleen parenchyma that cannot be characterized in this examination. No lytic or destructive lesions were detected in the bone structures within the image."
impressions="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Pericardial effusion, Emphysema, Atelectasis, Lung nodule, Pleural effusion, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_100_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.3',
               series_instance_uid='1.3.1',
               study_id='1',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.3',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.3', impressions)

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_100_a_1


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (90) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_100_a_1
Deleted Study: d9816a11-24e81c35-5665d21c-b96c57f2-5528d013
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.3 added or updated.


### Generated

In [42]:
prompt="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Pericardial effusion, Emphysema, Atelectasis, Lung nodule, Pleural effusion, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_100_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.3',
               series_instance_uid='1.3.2',
               study_id='2',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.3.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_100_a_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_100_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.3.2 added or updated.


### Counterfactual

In [43]:
prompt="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings."
description= "Pericardial effusion, Emphysema, Atelectasis, Lung nodule, Pleural effusion, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_100_a_1_counterf_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.3',
               series_instance_uid='1.3.3',
               study_id='3',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.3.3',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_100_a_1_counterf_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_100_a_1_counterf_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.3.3 added or updated.


# Example 4
- Lung nodule


### Original 

In [12]:
findings = "No lymph node was observed in the supraclavicular fossa and axilla in pathological size and appearance. Thyroid gland, left lobe and isthmus are operated. Heart dimensions and compartments appear natural. Pericardial effusion was not detected. Calibrations of mediastinal major vascular structures are natural. No pneumonic infiltration or consolidation area was detected in the lung parenchyma. No suspicious mass or nodular space-occupying lesion was observed in the lung parenchyma. There are a few nonspecific nodules less than 5 mm in diameter. In the evaluation of the upper abdominal organs, including the cross-section, there is a decrease in both kidney sizes and parenchyma thickness. No lytic-destructive lesions were detected in bone structures."
impressions="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/train/train_1000_a_1.nii.gz'


nifti_to_dicom(nifti_file=nifti_file,
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.4',
               series_instance_uid='1.4.1',
               study_id='1',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description
               )
store_metadata(series_instance_uid='1.4',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.4', impressions)

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1000_a_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1000_a_1
Deleted Study: 055e6b69-86797d7e-b58269a8-0bc35968-f2123fbd
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.4 added or updated.


## Generated

In [45]:
prompt="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1000_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1000_a_1/generated/'

nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.4',
               series_instance_uid='1.4.2',
               study_id='2',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.4.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1000_a_1_sample_0


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1000_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.4.2 added or updated.


### Counterfactual

In [46]:
prompt="Thoracic CT examination within normal limits. Increase in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1000_a_1_counterf_1_sample_0.nii.gz'


nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.4',
               series_instance_uid='1.4.3',
               study_id='3',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.4.3',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1000_a_1_counterf_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1000_a_1_counterf_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.4.3 added or updated.


# Example 5
- Arterial wall calcification
- Cardiomegaly
- Pericardial effusion
- Atelectasis
- Pleural effusion

### Original

In [13]:
findings = "The examination of the patient was evaluated by comparing it with the previous pulmonary CT angiography examination. The cardiothoracic ratio increased in favor of the heart. Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta. Trachea and both main bronchi are open. No occlusive pathology was detected in the trachea and both main bronchi. No pathologically enlarged lymph nodes were detected in the mediastinum and hilar regions.3.2021. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. In the upper abdominal organs within the sections, no mass with distinguishable borders was detected as far as it can be observed within the borders of non-enhanced CT. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Arterial wall calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.5',
               series_instance_uid='1.5.1',
               study_id='1',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description= description,
               )
store_metadata(series_instance_uid='1.5',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.5', impressions)


Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1001_a_1


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (94) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1
Deleted Study: ccf46c05-5156f3fd-a5277216-5ceffe85-236562f8
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.5 added or updated.


### Generated

In [48]:
prompt="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.2',
               study_id='2',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
store_metadata(series_instance_uid='1.5.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (190) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.5.2 added or updated.


### Counterfactual

In [49]:
prompt="Cardiomegaly, no pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.3',
               study_id='3',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
store_metadata(series_instance_uid='1.5.3',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.5.3 added or updated.


In [50]:
prompt="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 3 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.4',
               study_id='4',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
store_metadata(series_instance_uid='1.5.4',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_2_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.5.4 added or updated.


# Example 6
- Arterial wall calcification
- Coronary artery wall calcification
- Hiatal hernia
- Atelectasis
- Lung opacity
- Pulmonary fibrotic sequela
- Peribronchial thickening
- Consolidation
- Bronchiectasis

### Original

In [14]:
findings = "Trachea was in the midline of both main bronchi and no obstructive pathology was observed in the lumen. The mediastinum could not be evaluated optimally in the non-contrast examination. As far as can be seen; mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Diffuse atherosclerotic wall calcifications were observed in the thoracic aorta and coronary arteries. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. Sliding type hiatal hernia was observed at the lower end of the esophagus. No pathologically enlarged lymph nodes were detected in the mediastinum. No lymph node was observed in pathological size and appearance in the supraclavicular and axillary fossa. When examined in the lung parenchyma window; In both lungs, ground-glass densities including central-peripheral, crazy paving pattern and consolidation areas showing signs of vascular enlargement were observed. The outlook is consistent with Covid-19 pneumonia. The right hemidiaphragm is elevated. Diffuse linear subsegmental atelectatic changes were observed in the middle and lower lobes of the right lung. There are segmental-subsegmental bronchiectatic changes and peribronchial thickening in both lungs. Sequelae thickening was observed in the posterocostal pleura in both hemithorax. No mass lesion with distinguishable borders was detected in both lungs. A hypodense lesion area of 21x20 mm was observed at the junction of segment 4A-8 at the level of the liver dome and could not be characterized in this examination. The gallbladder was not observed (operated). The right adrenal gland and right kidney were not observed (operated). Diffuse thickening was observed in the left adrenal gland. Within the sections, free fluid-pathological lymph node was not observed in the abdomen. Mild dextroscoliosis with left opening was observed at the thoracic level. At mid-thoracic level, a bridging spur formation is observed in the right anterolateral corners of the vertebrae. Vertebral corpus heights are preserved."
impressions="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/train/train_101_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.6',
               series_instance_uid='1.6.1',
               study_id='1',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,
               )
store_metadata(series_instance_uid='1.6',metadata={"initial_findings":findings})
add_impression_metadata_to_study('1.6', impressions)

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_101_a_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_101_a_1
Deleted Study: 3711b673-3b11cd8a-5013519c-05e7448b-9dff183a
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.6 added or updated.


### Generated

In [52]:
prompt="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_101_a_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.6',
               series_instance_uid='1.6.2',
               study_id='2',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,

               )
store_metadata(series_instance_uid='1.6.2',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_101_a_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_101_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.6.2 added or updated.


# MedSyn Paper Examples

In [15]:
prompt="There is extensive consolidation seen. No evidence of cardiomegaly. No evidence of pleural effusion."
description="extensive consolidation v2"
nifti_file = '../data/nifti/MedSynExamples/extensive_consolidation_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.1',
               series_instance_uid='2.1.1',
               study_id='1',
               patient_name="MedSyn 1",
               patient_id="2_1",
               description=description,

               )
store_metadata(series_instance_uid='2.1.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/extensive_consolidation_v2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/extensive_consolidation_v2_sample_0
Deleted Study: 22f1c0dd-6d9004de-37fe2955-16ad4f9c-2a9cb1e0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.1.1 added or updated.


In [16]:
prompt="There are large pleural effusions seen. There is no airspace opacity or pneumothorax. There is no evidence of suspicious pulmonary nodule or mass."
description="large effusion v1"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.2',
               series_instance_uid='2.2.1',
               study_id='1',
               patient_name="MedSyn 2",
               patient_id="2_2",
               description=description,

               )
store_metadata(series_instance_uid='2.2.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/large_effusion_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/large_effusion_sample_0
Deleted Study: f87e50d6-dc9d391d-b9a1fcb0-9bceda2d-d146bfd9
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.2.1 added or updated.


In [17]:
prompt="There are large pleural effusions seen. No evidence of cardiomegaly. No evidence of consolidation."
description="large effusion v2"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.3',
               series_instance_uid='2.3.1',
               study_id='1',
               patient_name="MedSyn 3",
               patient_id="2_3",
               description=description,

               )
store_metadata(series_instance_uid='2.3.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/large_effusion_v2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/large_effusion_v2_sample_0
Deleted Study: a7086717-d4e3dfac-ec58aa30-59167982-60338c93
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.3.1 added or updated.


In [18]:
prompt="No bullae, cystic lung disease, or CT findings of small airways disease."
description="no bullae"
nifti_file = '../data/nifti/MedSynExamples/no_bullae_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.4',
               series_instance_uid='2.4.1',
               study_id='1',
               patient_name="MedSyn 4",
               patient_id="2_4",
               description=description,

               )
store_metadata(series_instance_uid='2.4.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/no_bullae_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/no_bullae_sample_0
Deleted Study: 424083c6-afec3296-ac18eb5f-af47c1cb-63926c4b
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.4.1 added or updated.


In [19]:
prompt="Emphysema is present with bullae. No cystic lung disease, or CT findings of small airways disease."
description="with bullae"
nifti_file = '../data/nifti/MedSynExamples/with_bullae_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.5',
               series_instance_uid='2.5.1',
               study_id='1',
               patient_name="MedSyn 5",
               patient_id="2_5",
               description=description,

               )
store_metadata(series_instance_uid='2.5.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/with_bullae_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/with_bullae_sample_0
Deleted Study: 23e2a27b-c8eb0c4b-6b5b3841-79e4808a-fa7824d7
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.5.1 added or updated.


In [20]:
prompt="There is no significant mediastinal lymphadenopathy. There is moderate cardiomegaly. The visualized upper abdominal organs are unremarkable. There is minimal perihepatic free fluid."
description="with cardiomegaly v2"
nifti_file = '../data/nifti/MedSynExamples/with_cardiomegaly_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.6',
               series_instance_uid='2.6.1',
               study_id='1',
               patient_name="MedSyn 6",
               patient_id="2_6",
               description=description,

               )
store_metadata(series_instance_uid='2.6.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/with_cardiomegaly_v2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/with_cardiomegaly_v2_sample_0
Deleted Study: 50bdc1e6-5cf3c2be-fb2d4437-d8b72733-448ae28f
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.6.1 added or updated.


In [21]:
prompt="Lungs: There is pulmonary fibrosis present with honeycombing demonstrated involving a significant portion of the bilateral lower lung zones. There are small areas of honeycombing in the anterior lateral right lung apex. There is no focal consolidation or pulmonary mass lesion demonstrated. There are a few areas of groundglass opacity in the right lung present, but the major finding is the pulmonary fibrosis that is most significant in the basilar regions. Pleural spaces:  There are small bilateral pleural effusions. There are no areas of significant pleural thickening demonstrated. Mediastinum and Lymph Nodes:  There is no pathologic adenopathy demonstrated in the axilla, mediastinum, or hilar regions. Heart and vascular structures: There is cardiomegaly. There is a cardiac pacemaker in place. There are small areas of atherosclerotic disease present, likely to include in the coronary arteries. There is no aneurysmal dilatation of the thoracic aorta. There is no significant pericardial effusion. Esophagus and visualized portion of the gastrointestinal tract: There is a mildly dilated partially fluid-filled distal esophagus. This could represent a hiatal hernia. Osseous structures and chest wall:  Unremarkable without acute or significant non-degenerative abnormalities. Visualized portion of the lower neck:  No major abnormalities are demonstrated in the portion of the neck included on this chest CT scan. Visualized portion of the upper abdomen:  There is ascites present with a small amount of fluid surrounding the liver and spleen. There is significant pulmonary fibrosis present, with large areas of honeycombing present involving both lower lung zones. There are milder areas of abnormality in the subpleural right greater than left upper lung zones. There are mild areas of groundglass opacity in the right upper lung zone, but there is no consolidation in the lungs. There are no suspicious pulmonary mass lesions. There are very small bilateral pleural effusions. There is at least mild ascites seen in the portion of the upper abdomen included on this exam."
description="with cardiomegaly v3"
nifti_file = '../data/nifti/MedSynExamples/with_cardiomegaly_v3_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.7',
               series_instance_uid='2.7.1',
               study_id='1',
               patient_name="MedSyn 7",
               patient_id="2_7",
               description=description,

               )
store_metadata(series_instance_uid='2.7.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/MedSynExamples/with_cardiomegaly_v3_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/MedSynExamples/with_cardiomegaly_v3_sample_0
Deleted Study: bee97093-23cfa59f-e4b506d1-0f1871b6-c2c6d630
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 2.7.1 added or updated.


# Random generated


In [53]:
prompt= 'Left lobectomy'
description=""
nifti_file = '../data/nifti/examples/left_leb_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated", 
               modality = "AI",
               study_instance_uid='1.7',
               series_instance_uid='1.7.1',
               study_id='1',
               patient_name="Example 7",
               patient_id="generated_1",
               description=description,

               )
store_metadata(series_instance_uid='1.7.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/examples/left_leb_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/left_leb_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.7.1 added or updated.


In [60]:
prompt= 'Left pneumothorax'
description=""
nifti_file = '../data/nifti/examples/left_pneu_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated", 
               modality = "CT",
               study_instance_uid='1.8',
               series_instance_uid='1.8.1',
               study_id='1',
               patient_name="Example 8",
               patient_id="generated_2",
               description=description,

               )
store_metadata(series_instance_uid='1.8.1',metadata={"impressions":prompt})

Store DICOM files in Folder: ../data/nifti/examples/left_pneu_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/left_pneu_sample_0
Deleted Study: 94d4dd7f-c072e8b6-0d430a66-ee67cd26-35a79395
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
Entry with study_instance_uid 1.8.1 added or updated.
